In [26]:
# Import Required Libs
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np
import requests
from qdrant_client.http.models import PointStruct, Filter
from fastembed.embedding import DefaultEmbedding
import numpy as np

In [11]:
# Load the model
model_handle = DefaultEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Embeding the query
query = "I just discovered the course. Can I join now?"

# Generate embedding
embedding_vector = next(model_handle.embed(query))  # Returns a generator, use next()

# Create PointStruct with payload
query_point = models.PointStruct(
    id=0,
    vector=embedding_vector,
    payload={
        "text": query
    }
)

# Print the minimum value in the vector
print("Embedding vector length:", len(embedding_vector))
print("Minimum value in vector:", min(embedding_vector))

Embedding vector length: 512
Minimum value in vector: -0.11726373885183883


In [14]:
# Consine Similarity
texts = [
    "I just discovered the course. Can I join now?",
    "Can I still join the course after the start date?"
]
vectors = list(model_handle.embed(texts))

query_vector = vectors[0]
doc_vector = vectors[1]

# Since vectors are normalized, cosine similarity = dot product
cosine_sim = np.dot(query_vector, doc_vector)
print("Cosine similarity:", cosine_sim)

Cosine similarity: 0.9008528895674548


In [16]:
# Documents
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

# Embed query and documents
doc_texts = [doc['text'] for doc in documents]
doc_vectors = np.array(list(model_handle.embed(doc_texts)))

# Compute cosine similarities
similarities = doc_vectors.dot(query_vector)

# Find the index with highest similarity
best_index = np.argmax(similarities)

print("Cosine similarities:", similarities)
print("Most similar document index:", best_index)

Cosine similarities: [0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]
Most similar document index: 1


In [22]:
# Full text
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

# Query Embedding
query_embedding = list(model_handle.embed([query]))[0]

full_text_embeddings = np.array(list(model_handle.embed(full_texts)))
cosine_similarities = full_text_embeddings.dot(query_embedding)
best_doc_index = np.argmax(cosine_similarities)

print("Cosine similarities for each document:", cosine_similarities)
print("Most similar document index:", best_doc_index)

Cosine similarities for each document: [0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]
Most similar document index: 0


In [31]:
# Smallest Model
small_model = DefaultEmbedding(model_name="BAAI/bge-small-en")

# Example embedding
vec = list(small_model.embed(["Test sentence"]))[0]
print("Embedding dimension:", len(vec)) 
EMBEDDING_DIMENSIONALITY = len(vec)

Embedding dimension: 384


In [27]:
# No.6
# Step 1: Download raw documents
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [28]:
documents = []
for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [29]:
model_handle = DefaultEmbedding(model_name="BAAI/bge-small-en")

In [32]:
client = QdrantClient()

collection_name = "ml_zoomcamp_faq"

client = QdrantClient()

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [33]:
points = []
for idx, doc in enumerate(documents):
    full_text = doc['question'] + " " + doc['text']
    vector = list(model_handle.embed([full_text]))[0]
    point = PointStruct(
        id=idx,
        vector=vector,
        payload={
            "question": doc['question'],
            "text": doc['text'],
            "course": doc['course']
        }
    )
    points.append(point)

In [34]:
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
query_text = "I just discovered the course. Can I join now?"
query_vector = list(model_handle.embed([query_text]))[0]

results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
    with_payload=True
)

/tmp/ipykernel_19445/1145625625.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [36]:
print("Highest score from search results:", results[0].score)

Highest score from search results: 0.8703172
